In [15]:
from utils import *

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

from joblib import Parallel, delayed

In [2]:
# Rutas de los datos
ruta_entrenamiento = "db/KDDTrain+.arff"
ruta_prueba = "db/KDDTest+.arff"
ruta_entrenamiento, ruta_prueba

('db/KDDTrain+.arff', 'db/KDDTest+.arff')

In [3]:
X_entrenamiento, y_entrenamiento = procesar_datos_entrada(ruta_entrenamiento)
X_prueba, y_prueba = procesar_datos_entrada(ruta_prueba)

print("Dimensiones de las características de entrenamiento:",
    X_entrenamiento.shape)
print("Dimensiones de las características de prueba:", X_prueba.shape)

Dimensiones de las características de entrenamiento: (125973, 41)
Dimensiones de las características de prueba: (22544, 41)


In [4]:
preprocesador = obtener_preprocesador(X_entrenamiento)
preprocesador

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rate',
       'dst_host_srv_serror_rate', 'du...
       'num_outbound_cmds', 'num_root', 'num_shells', 'rerror_rate',
       'root_shell', 'same_srv_rate', 'serror_rate', 'src_bytes', 'srv_count',
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])

In [5]:
modelos = {
    'Regresión logística': {
        'modelo': LogisticRegression(max_iter=1000),
        'params': {
            'regresor__C': [0.01, 0.1, 1.0, 10.0, 100.0],
            'regresor__penalty': ['l2']
        }
    },
    'KNN': {
        'modelo': KNeighborsClassifier(),
        'params': {
            'regresor__n_neighbors': [3, 5, 7, 9],
            'regresor__weights': ['uniform', 'distance']
        }
    },
    'Árbol de decisión': {
        'modelo': DecisionTreeClassifier(),
        'params': {
            'regresor__max_depth': [3, 5, 7, None],
            'regresor__min_samples_split': [2, 5, 10],
            'regresor__min_samples_leaf': [1, 2, 4]
        }
    },
    'SVM': {
        'modelo': SVC(),
        'params': {
            'regresor__C': [0.1, 1.0, 10.0, 100.0],
            'regresor__kernel': ['linear', 'rbf']
        }
    },
    'Bosque aleatorio': {
        'modelo': RandomForestClassifier(),
        'params': {
            'regresor__n_estimators': [100, 200, 300],
            'regresor__max_features': ['sqrt', 'log2', None],
            'regresor__max_depth': [None, 10, 20, 30],
            'regresor__min_samples_split': [2, 5, 10],
            'regresor__min_samples_leaf': [1, 2, 4]
        }
    },
    'Potenciación de gradiente': {
        'modelo': GradientBoostingClassifier(),
        'params': {
            'regresor__n_estimators': [100, 200, 300],
            'regresor__learning_rate': [0.01, 0.1, 0.2],
            'regresor__max_depth': [3, 4, 5],
            'regresor__min_samples_split': [2, 5, 10],
            'regresor__min_samples_leaf': [1, 2, 4]
        }
    }
}

In [16]:
# Configurar la validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluar_modelo(nombre, modelo):
    pipeline = Pipeline(steps=[
        ('preprocesador', preprocesador),
        ('clasificador', modelo['modelo'])
    ])

    # Configurar búsqueda de hiperparámetros
    grid_search = GridSearchCV(pipeline, modelo['params'], cv=kf,
        scoring='recall', n_jobs=-1)
    
    # Realizar la búsqueda de hiperparámetros
    grid_search.fit(X_entrenamiento, y_entrenamiento)

    # Imprimir los resultados
    print(f"{nombre}: Mejor sensibilidad = {grid_search.best_score_:.4f}")
    print(f"{nombre}: Mejores parámetros = {grid_search.best_params_}")
    print()

    # Guardar los mejores resultados
    return nombre, {
        'mejor_sensibilidad': grid_search.best_score_,
        'mejores_params': grid_search.best_params_
    }

In [52]:
# Ejecutar la evaluación de los modelos en paralelo
resultados = Parallel(n_jobs=-1)(
    delayed(evaluar_modelo)(nombre, modelo)
        for nombre, modelo in modelos.items()
)

Regresión logística: Mejor sensibilidad = 0.9538
Regresión logística: Mejores parámetros = {'regresor__C': 10.0, 'regresor__penalty': 'l2'}

SVM: Mejor sensibilidad = 0.9559
SVM: Mejores parámetros = {'regresor__C': 100.0, 'regresor__kernel': 'rbf'}

KNN: Mejor sensibilidad = 0.9289
KNN: Mejores parámetros = {'regresor__n_neighbors': 3, 'regresor__weights': 'distance'}

Árbol de decisión: Mejor sensibilidad = 0.9615
Árbol de decisión: Mejores parámetros = {'regresor__max_depth': 7, 'regresor__min_samples_leaf': 1, 'regresor__min_samples_split': 2}



/usr/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Potenciación de gradiente: Mejor sensibilidad = 0.9842
Potenciación de gradiente: Mejores parámetros = {'regresor__learning_rate': 0.1, 'regresor__max_depth': 4, 'regresor__min_samples_leaf': 1, 'regresor__min_samples_split': 2, 'regresor__n_estimators': 100}

Bosque aleatorio: Mejor sensibilidad = 0.9842
Bosque aleatorio: Mejores parámetros = {'regresor__max_depth': None, 'regresor__max_features': None, 'regresor__min_samples_leaf': 1, 'regresor__min_samples_split': 2, 'regresor__n_estimators': 200}

[None, None, None, None, None, None]


In [7]:
# Configurar la tubería con los mejores parámetros
pipeline = Pipeline(steps=[
    ('preprocesador', preprocesador),
    ('clasificador', RandomForestClassifier(
        max_depth=None,
        max_features=None,
        min_samples_leaf=1,
        min_samples_split=2,
        n_estimators=200
    ))
])

In [8]:
# Ajustar el modelo a los datos de entrenamiento
pipeline.fit(X_entrenamiento, y_entrenamiento)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rat...
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])),
                ('clasificador',
                 RandomForestClassifier(max_features=None, n_estimators=200))])

In [9]:
# Evaluar el modelo con los datos de prueba
y_pred = pipeline.predict(X_prueba)
y_pred[:5]

array([1, 1, 0, 1, 0])

In [12]:
# Evaluar la sensibilidad en los datos de prueba
exactitud = accuracy_score(y_prueba, y_pred)
precision = precision_score(y_prueba, y_pred)
sensibilidad = recall_score(y_prueba, y_pred)
f1 = f1_score(y_prueba, y_pred)
area_roc = roc_auc_score(y_prueba, y_pred)

print(f"Exactitud en los datos de prueba: {exactitud:.4f}")
print(f"Precisión en los datos de prueba: {precision:.4f}")
print(f"Sensibilidad en los datos de prueba: {sensibilidad:.4f}")
print(f"Métrica F1 en los datos de prueba: {f1:.4f}")
print(f"Área bajo la curva ROC en los datos de prueba: {area_roc:.4f}")


Exactitud en los datos de prueba: 0.8038
Precisión en los datos de prueba: 0.9687
Sensibilidad en los datos de prueba: 0.6772
Métrica F1 en los datos de prueba: 0.7971
Área bajo la curva ROC en los datos de prueba: 0.8241


In [17]:
# Guardar los resultados
pd.Series(y_pred).to_csv('db/resultados_clasificacion.csv', index=False,
    header=False)